In [17]:
import os
import zipfile

def zip_folders(folder_path, zip_filename):
    with zipfile.ZipFile(zip_filename, 'w', zipfile.ZIP_DEFLATED) as zipf:
        for root, _, files in os.walk(folder_path):
            for file in files:
                file_path = os.path.join(root, file)
                # Calculate the relative path to the file inside the zip
                rel_path = os.path.relpath(file_path, os.path.dirname(folder_path))
                zipf.write(file_path, rel_path)

In [18]:
# Example usage
root = "/home/jupyter/ai_font/data/reports/exp0514"
fd = "cggan"
zip_folders(f"{root}/{fd}", f"{fd}.zip")

In [19]:
from google.cloud import storage

def upload_to_bucket(bucket_name, source_file_name, destination_blob_name):
    """Uploads a file to a Google Cloud Storage bucket."""
    # Initialize a client
    storage_client = storage.Client()
    
    # Get the bucket
    bucket = storage_client.bucket(bucket_name)
    
    # Upload the file
    blob = bucket.blob(destination_blob_name)
    blob.upload_from_filename(source_file_name)
    
    print(f"File {source_file_name} uploaded to {bucket_name} as {destination_blob_name}")

In [20]:
# Example usage
bucket_name = "leo_font"
destination_blob_name = f"zipfiles/{fd}.zip"

upload_to_bucket(bucket_name, f"{fd}.zip", destination_blob_name)

File cggan.zip uploaded to leo_font as zipfiles/cggan.zip
